## SQLAlchemy - using an ORM

### what is an ORM?

ORM stands for Object Relational Mapper

https://en.wikipedia.org/wiki/Object-relational_mapping

```
The heart of the problem involves translating the logical representation 
of the objects into an atomized form that is capable of being stored 
in the database, while preserving the properties of the objects 
and their relationships so that they can be reloaded as objects when needed.
```

### what is SQLAlchemy?

* SQLAlchemy is magic!!  

* Allows you to make database style calls in Python as opposed to SQL.

* Can connect to lots of differnt databases.  Underlying connection engine for many other Python packages like pandas.

* Can use raw SQL if you want

* Allows you to have an abstraction layer between Python and database

https://www.sqlalchemy.org/

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html#pandas.read_sql

### install SQLAlchemy

```pip install SQLAlchemy```

### make sure it works

In [36]:
import sqlalchemy
sqlalchemy.__version__

'1.0.6'

### create a simple database in memory

In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [5]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### define class representing a simple table

In [7]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

In [8]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

### create the database!!

In [9]:

Base.metadata.create_all(engine)

2017-04-11 18:05:20,454 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-11 18:05:20,454 INFO sqlalchemy.engine.base.Engine ()
2017-04-11 18:05:20,455 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-11 18:05:20,456 INFO sqlalchemy.engine.base.Engine ()
2017-04-11 18:05:20,457 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-11 18:05:20,457 INFO sqlalchemy.engine.base.Engine ()
2017-04-11 18:05:20,458 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-04-11 18:05:20,459 INFO sqlalchemy.engine.base.Engine ()
2017-04-11 18:05:20,460 INFO sqlalchemy.engine.base.Engine COMMIT


### create our first User

In [10]:

amit_user = User(name='amit', fullname='Amit Bhattacharyya', password='gobears')

### __repr__ should return a printable representation of the object

In [11]:
# use the defined __repr__ function

__repr__ should return a printable representation of the object
amit_user

<User(name='amit', fullname='Amit Bhattacharyya', password='gobears')>

In [12]:
amit_user.password

'gobears'

In [13]:
amit_user.fullname

'Amit Bhattacharyya'

### need to start a Session in order to "talk to the database"

so far we have only created an object in Python memory using the class we defined.

In [17]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session() # instance of Session()

### first insert 

In [24]:
session.add(amit_user)
session.commit()

2017-04-11 18:17:23,531 INFO sqlalchemy.engine.base.Engine COMMIT


### query to see if it is there

In [21]:
our_user = session.query(User)
our_user.first()

2017-04-11 18:15:20,342 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-11 18:15:20,344 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-11 18:15:20,344 INFO sqlalchemy.engine.base.Engine ('amit', 'Amit Bhattacharyya', 'gobears')
2017-04-11 18:15:20,346 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
 LIMIT ? OFFSET ?
2017-04-11 18:15:20,346 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(name='amit', fullname='Amit Bhattacharyya', password='gobears')>

### query with a filter

In [27]:
our_user = session.query(User).filter_by(name='amit')
our_user.count()

2017-04-11 18:17:42,693 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2017-04-11 18:17:42,694 INFO sqlalchemy.engine.base.Engine ('amit',)


1

### add a bunch more users


In [28]:

aryana = User(name='aryana', fullname='Aryana Bhattacharyya', password='swimmie')
anika = User(name='anika', fullname='Anika Bhattacharyya', password='feather')
session.add_all([ aryana, anika ] )
session.commit()

2017-04-11 18:19:25,138 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-11 18:19:25,139 INFO sqlalchemy.engine.base.Engine ('aryana', 'Aryana Bhattacharyya', 'swimmie')
2017-04-11 18:19:25,139 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-11 18:19:25,140 INFO sqlalchemy.engine.base.Engine ('anika', 'Anika Bhattacharyya', 'feather')
2017-04-11 18:19:25,141 INFO sqlalchemy.engine.base.Engine COMMIT


### check how many Users are in table

In [29]:
query = session.query(User)
query.count()

2017-04-11 18:19:50,185 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-11 18:19:50,186 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users) AS anon_1
2017-04-11 18:19:50,187 INFO sqlalchemy.engine.base.Engine ()


3

### try raw SQL

In [33]:

from sqlalchemy import text

sql = text('select count(*) from users')
result = session.execute(sql)
result.first()

2017-04-11 18:24:48,903 INFO sqlalchemy.engine.base.Engine select count(*) from users
2017-04-11 18:24:48,904 INFO sqlalchemy.engine.base.Engine ()


(3,)

In [34]:
sql = text('select * from users')
result = session.execute(sql)
for rr in result:
    print rr

2017-04-11 18:25:36,286 INFO sqlalchemy.engine.base.Engine select * from users
2017-04-11 18:25:36,287 INFO sqlalchemy.engine.base.Engine ()
(1, u'amit', u'Amit Bhattacharyya', u'gobears')
(2, u'aryana', u'Aryana Bhattacharyya', u'swimmie')
(3, u'anika', u'Anika Bhattacharyya', u'feather')
